##Reproduction of the results in the paper "Good Agreements make good friends" Anh et al. 2013

In [1]:
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline
import os

In [2]:
import egttools as egt

In [3]:
egt.Random.init()
seed = egt.Random._seed

In [4]:
# Payoff matrix
R=1;P=0;T=2;S=-1
beta=0.1
delta=4
eps=0.25

A = np.array([
        [ R-(eps/2), R-eps,0,S+delta-eps,R-eps],
        [R,R,S,S,S],
        [0,T,P,P,P],
        [T-delta,T,P,P,P],
        [R,T,P,P,P],
    ])

In [5]:
A

array([[ 0.875,  0.75 ,  0.   ,  2.75 ,  0.75 ],
       [ 1.   ,  1.   , -1.   , -1.   , -1.   ],
       [ 0.   ,  2.   ,  0.   ,  0.   ,  0.   ],
       [-2.   ,  2.   ,  0.   ,  0.   ,  0.   ],
       [ 1.   ,  2.   ,  0.   ,  0.   ,  0.   ]])

In [6]:
strategy_labels=['Commit','C','D','Fake','Free']

In [7]:
strategy_labels

['Commit', 'C', 'D', 'Fake', 'Free']

In [ ]:
game = egt.games.Matrix2PlayerGameHolder(5, A)

In [ ]:
Z=100
evolver = egt.analytical.PairwiseComparison(Z, game)

In [ ]:
beta=0.1
transition_matrix,fixation_probabilities = evolver.calculate_transition_and_fixation_matrix_sml(beta)
stationary_distribution = egt.utils.calculate_stationary_distribution(transition_matrix.transpose())
stationary_distribution

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5), dpi=150)
G = egt.plotting.draw_invasion_diagram(strategy_labels,
                                              1/Z, fixation_probabilities, stationary_distribution,
                                              node_size=1000,
                                              font_size_node_labels=8,
                                              font_size_edge_labels=8,
                                              font_size_sd_labels=8,
                                              edge_width=1,
                                              min_strategy_frequency=0.00001,
                                              ax=ax)
plt.axis('off')
plt.show() # display

In [ ]:
from tqdm import tqdm
epss=np.linspace(0, 1.5, 30)
sds=[]

for eps in tqdm(epss):
    A = np.array([
        [ R-(eps/2), R-eps,0,S+delta-eps,R-eps],
        [R,R,S,S,S],
        [0,T,P,P,P],
        [T-delta,T,P,P,P],
        [R,T,P,P,P],
    ])
    game = egt.games.Matrix2PlayerGameHolder(5, A)
    evolver = egt.analytical.PairwiseComparison(Z, game)
    transition_matrix,fixation_probabilities = evolver.calculate_transition_and_fixation_matrix_sml(beta)
    stationary_distribution = egt.utils.calculate_stationary_distribution(transition_matrix.transpose())
    sds.append(stationary_distribution)

sds = np.asarray(sds)
sds

In [ ]:
import pandas as pd
import seaborn as sns

labels = ['epsilon'] + strategy_labels
markers = ['o', 'x', 'v', '^', 'X', '*', 'h', '1']
#colors=colors = sns.color_palette("icefire", 8)

fig, ax = plt.subplots(figsize=(7, 5))

for i in range(5):
    ax.plot(epss, sds[:,i],linewidth=2,label=strategy_labels[i], marker=markers[i])

ax = plt.gca()
ax.set_ylim(-0.01, 0.9)
ax.set_ylabel('Stationary distribution', fontsize=14)
ax.set_xlabel('Commitment cost $\epsilon$', fontsize=14)
plt.legend(title="Strategy", loc='upper right', bbox_to_anchor=(1.4, 1), frameon=False, fontsize=12,
          title_fontsize=12)

plt.tight_layout()
plt.show()
fig.savefig(
    f"./commitment_delta_{delta}_Z_{Z}_beta_{beta}.png",
    bbox_inches='tight', dpi=300)